In [801]:

def z_factor(product1,product2):
    z=1
    y=1
    factor=1000
    if product1=='RB' and product2 =='RB':
        z=42
        y=42
    elif product1=='HO' and product2 =='HO':
        z=42
        y=42
    elif product1=='RB' and product2 =='HO':
        z=42
        y=42
    elif product1=='HO' and product2 =='RB':
        z=42
        y=42
    elif product1=='G' and product2 =='G':
        factor=100
    elif product1=='G' and product2 =='B':
        z=(1/7.45)
    elif product1=="HO" and product2=='G':
        z=42
        y=(1/7.45)
    
    else:
        z=1
        y=1
        factor=1000
    
    return z,y, factor
        
def create_sprd(dataframe1, dataframe2):
   
    dataframe1.reset_index(inplace=True)
    dataframe2.reset_index(inplace=True)
    date=dataframe1['Date']
    
    dataframe1.drop(columns=['Date'],inplace=True)
    dataframe2.drop(columns=['Date'],inplace=True)
    
    while True:
        sprd_type=int(raw_input('Please indicate if this is a 1) Crack or 2) Spread - please input integer 1 or 2: '))
        
        if sprd_type==1:
            
            new_df=dataframe1*42-dataframe2
            new_df['Date']=date
            new_df.set_index('Date',inplace=True)
            new_df.fillna(method='ffill',inplace=True)
            new_df.fillna(method='bfill',inplace=True)
            
            return new_df
            break
        
        elif sprd_type==2:
            
            new_df= dataframe1-dataframe2
            new_df['Date']=date
            new_df.set_index('Date',inplace=True)
            new_df.fillna(method='ffill',inplace=True)
            new_df.fillna(method='bfill',inplace=True)
            
            return new_df
            break
        
        else:
            print 'You have inputted incorrectly. Please try again and enter either integer 1 or 2.'
    

class Mth_1(object):
    
    def __init__(self):
        pass
        
    def pdt_type1(self):
        
        product=''
        while True:
            inst=int(raw_input('Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: '))
    
            if inst==1:
                product='B'
                break
            elif inst==2:
                product='G'
                break   
            elif inst==3:
                product='HO'
                break      
            elif inst==4:
                product='RB'
                break       
            elif inst==5:
                product='CL'
                break       
            else:
                print 'You did not input a correct number. Please try again.'
        
        return product
        
    
    def mth_no1(self):
        month=''
        while True:
            Mth=int(raw_input('Please input the month of the contract which you wish to evaluate in numbers from 1-12: '))
    
            if Mth==1:
                month='Jan'
                break
            elif Mth==2:
                month='Feb'
                break   
            elif Mth==3:
                month='Mar'
                break   
            elif Mth==4:
                month='Apr'
                break   
            elif Mth==5:
                month='May'
                break          
            elif Mth==6:
                month='Jun'
                break   
            elif Mth==7:
                month='Jul'
                break   
            elif Mth==8:
                month='Aug'
                break   
            elif Mth==9:
                month='Sep'
                break   
            elif Mth==10:
                month='Oct'
                break   
            elif Mth==11:
                month='Nov'
                break   
            elif Mth==12:
                month='Dec'
                break 
            else:
                print 'You did not input a correct number. Please try again.'
        
        return month

    def dataframe_create(self,start_mth,start_day,end_mth,end_day,i,j):

        count=0
        path = os.getcwd()
        
        listing = os.listdir(path)
        for infile in listing:
            temp=pd.read_excel(infile) #read each file into panda dataframe
    
            contract_yr_str=infile[-8:-4] # get the contract year
    
            seasonal_year=int(contract_yr_str)-1 # get the seasonal year to create the index - prev year
            seasonal_year=str(seasonal_year) # convert to string

            start_date=str(int(seasonal_year)+i)+'-'+start_mth+'-'+start_day # create start date
            end_date=str(int(seasonal_year)+j)+'-'+end_mth+'-'+end_day # create end date
            
            #Join a date dataframe and the temp dataframe together
            
            dates=pd.date_range(start_date, end_date) # range of dates for date axis
            dates=dates.strftime('%Y-%m-%d') #make sure no left out dates
            sample_data=np.arange(0,len(dates)) #just sample numbers to create dataframe
    
            dates_dataframe=pd.DataFrame({'Date':dates,'data':sample_data}) # create dates dataframe
            dates_dataframe=dates_dataframe.set_index('Date') # set index as Date to facilitate joining
    
            temp=temp.set_index('Date') # set index as Date to facilitate joining
    
            temp=dates_dataframe.join(temp) # join dataframes together on Date
               
            #filter dates by the start and end date
            temp.reset_index(inplace=True)
    
            temp=temp[temp['Date']>=start_date] # filter for start date

            temp=temp[temp['Date']<=end_date] # filter for end date

            temp['Date']=pd.to_datetime(temp['Date']).dt.strftime('%m/%d') # putting date in mm/dd form
            temp=temp.set_index('Date') # putting date back as index
            temp=pd.DataFrame(temp['Settle'],columns=['Settle']) # setting dataframe with date and settle only

    
            if count==0:
                Seasonal_Combi=temp
                seasonal_year=str(int(seasonal_year)+1)
                Seasonal_Combi.rename(columns={'Settle':seasonal_year},inplace=True)
        
                count+=1
        
            else:
    
                seasonal_year=str(int(seasonal_year)+1)
                Seasonal_Combi['Settle']=temp['Settle']
                Seasonal_Combi.rename(columns={'Settle':seasonal_year},inplace=True)

        #don't need this, i believe
        #Seasonal_Combi.drop(columns=['Open','High','Low','Last','Change', 'Volume','Prev. Day Open Interest'],inplace=True)   

        
        return Seasonal_Combi
            
            

class Mth_2(object):
    
    def __init__(self):
        pass
        
    def pdt_type2(self):
        
        product=''
        while True:
            inst=int(raw_input('Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: '))
    
            if inst==1:
                product='B'
                break
            elif inst==2:
                product='G'
                break   
            elif inst==3:
                product='HO'
                break      
            elif inst==4:
                product='RB'
                break       
            elif inst==5:
                product='CL'
                break       
            else:
                print 'You did not input a correct number. Please try again.'
        
        return product
        
    
    def mth_no2(self):
        month=''
        while True:
            Mth=int(raw_input('Please input the month of the contract which you wish to evaluate in numbers from 1-12: '))
    
            if Mth==1:
                month='Jan'
                break
            elif Mth==2:
                month='Feb'
                break   
            elif Mth==3:
                month='Mar'
                break   
            elif Mth==4:
                month='Apr'
                break   
            elif Mth==5:
                month='May'
                break          
            elif Mth==6:
                month='Jun'
                break   
            elif Mth==7:
                month='Jul'
                break   
            elif Mth==8:
                month='Aug'
                break   
            elif Mth==9:
                month='Sep'
                break   
            elif Mth==10:
                month='Oct'
                break   
            elif Mth==11:
                month='Nov'
                break   
            elif Mth==12:
                month='Dec'
                break 
            else:
                print 'You did not input a correct number. Please try again.'
        
        return month

    def dataframe_create(self, start_mth, start_day, end_mth,end_day,i,j):

        count=0
        path = os.getcwd()

        listing = os.listdir(path)
        for infile in listing:
            temp=pd.read_excel(infile) #read each file into panda dataframe
    
            contract_yr_str=infile[-8:-4] # get the contract year
    
            seasonal_year=int(contract_yr_str)-1 # get the seasonal year to create the index - prev year
            seasonal_year=str(seasonal_year) # convert to string
     
            start_date=str(int(seasonal_year)+i)+'-'+start_mth+'-'+start_day # create start date
            end_date=str(int(seasonal_year)+j)+'-'+end_mth+'-'+end_day # create end date
            
            #Join a date dataframe and the temp dataframe together
            
            dates=pd.date_range(start_date, end_date) # range of dates for date axis
            dates=dates.strftime('%Y-%m-%d') #make sure no left out dates
            sample_data=np.arange(0,len(dates)) #just sample numbers to create dataframe
    
            dates_dataframe=pd.DataFrame({'Date':dates,'data':sample_data}) # create dates dataframe
            dates_dataframe=dates_dataframe.set_index('Date') # set index as Date to facilitate joining
    
            temp=temp.set_index('Date') # set index as Date to facilitate joining
    
            temp=dates_dataframe.join(temp) # join dataframes together on Date
               
            #filter dates by the start and end date
            temp.reset_index(inplace=True)
    
            temp=temp[temp['Date']>=start_date] # filter for start date

            temp=temp[temp['Date']<=end_date] # filter for end date

            temp['Date']=pd.to_datetime(temp['Date']).dt.strftime('%m/%d') # putting date in mm/dd form
            temp=temp.set_index('Date') # putting date back as index
            temp=pd.DataFrame(temp['Settle'],columns=['Settle']) # setting dataframe with date and settle only
    
            if count==0:
                Seasonal_Combi=temp
                seasonal_year=str(int(seasonal_year)+1)
                Seasonal_Combi.rename(columns={'Settle':seasonal_year},inplace=True)
        
                count+=1
        
            else:
    
                seasonal_year=str(int(seasonal_year)+1)
                Seasonal_Combi['Settle']=temp['Settle']
                Seasonal_Combi.rename(columns={'Settle':seasonal_year},inplace=True)

        #Seasonal_Combi.drop(columns=['Open','High','Low','Last','Change','Volume','Prev. Day Open Interest'],inplace=True)   

        
        return Seasonal_Combi
    

        
    
import numpy as np
import pandas as pd
import os
import statsmodels.tsa.stattools as ts
import matplotlib.pyplot as plt  
%matplotlib inline

#user specify instrument and contract month

product1=''
month1=''
product2=''
month2=''

front_mth=Mth_1()
product1 = front_mth.pdt_type1()
month1=front_mth.mth_no1()

back_mth=Mth_2()
product2=back_mth.pdt_type2()
month2=back_mth.mth_no2()

        

Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: 5
Please input the month of the contract which you wish to evaluate in numbers from 1-12: 8
Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: 1
Please input the month of the contract which you wish to evaluate in numbers from 1-12: 8


In [802]:
cd Quandl

C:\Users\boonsin.teo\Documents\Quandl


In [803]:
cd $product1

C:\Users\boonsin.teo\Documents\Quandl\CL


In [804]:
cd $month1

C:\Users\boonsin.teo\Documents\Quandl\CL\Aug


In [805]:
start_mth=raw_input('Please input start mth of seasonal period in MM format: ')
start_day=raw_input('Please input start day of seasonal period in DD format: ')
i=int(raw_input('Please input 1 if start mth is the same as contract year and 0 if not: '))
            
end_mth=raw_input('Please input end mth of seasonal period in MM format: ')
end_day=raw_input('Please input end day of seasonal period in DD format: ')
            
j=int(raw_input('Please input 1 if end mth is the same as contract year and 0 if not: '))


front_month=front_mth.dataframe_create(start_mth,start_day,end_mth,end_day,i,j)

Please input start mth of seasonal period in MM format: 08
Please input start day of seasonal period in DD format: 01
Please input 1 if start mth is the same as contract year and 0 if not: 0
Please input end mth of seasonal period in MM format: 06
Please input end day of seasonal period in DD format: 30
Please input 1 if end mth is the same as contract year and 0 if not: 1


In [806]:
cd ..

C:\Users\boonsin.teo\Documents\Quandl\CL


In [807]:
cd ..

C:\Users\boonsin.teo\Documents\Quandl


In [808]:
cd ..

C:\Users\boonsin.teo\Documents


In [809]:
cd Quandl

C:\Users\boonsin.teo\Documents\Quandl


In [810]:
cd $product2

C:\Users\boonsin.teo\Documents\Quandl\B


In [811]:
cd $month2

C:\Users\boonsin.teo\Documents\Quandl\B\Aug


In [812]:
back_month=back_mth.dataframe_create(start_mth, start_day,end_mth,end_day,i,j)

In [813]:
cd ..

C:\Users\boonsin.teo\Documents\Quandl\B


In [814]:
cd ..

C:\Users\boonsin.teo\Documents\Quandl


In [815]:
cd ..

C:\Users\boonsin.teo\Documents


In [816]:
front_month.head()

,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Date,,,,,,,,,,,,,,,,,,,,,
08/01,17.31,18.35,20.14,NaN,NaN,NaN,24.42,24.21,27.16,NaN,...,125.97,NaN,NaN,99.44,91.25,95.17,92.58,NaN,45.78,49.89
08/02,17.34,18.40,NaN,NaN,18.76,NaN,24.83,24.55,NaN,39.12,...,NaN,NaN,85.80,98.32,89.71,95.04,NaN,NaN,45.40,50.22
08/03,17.33,NaN,NaN,16.08,18.72,NaN,24.87,NaN,NaN,39.58,...,NaN,79.19,86.80,96.51,93.14,NaN,NaN,51.20,46.36,49.76
08/04,17.36,NaN,20.56,16.14,18.81,NaN,NaN,NaN,26.95,38.44,...,122.09,79.88,86.69,91.52,NaN,NaN,93.08,51.47,47.17,50.25
08/05,NaN,18.31,20.50,16.13,18.91,NaN,NaN,24.34,27.15,39.47,...,119.12,80.57,86.21,91.98,NaN,95.23,92.82,50.98,47.17,NaN


In [817]:
back_month.head()

,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Date,,,,,,,,,,,,,,,,,,,,,
08/01,16.13,17.19,18.83,NaN,NaN,24.14,23.23,23.33,26.10,NaN,...,129.06,NaN,NaN,115.67,101.47,102.68,105.09,NaN,46.65,52.82
08/02,16.18,17.12,NaN,NaN,17.42,24.23,23.72,23.59,NaN,36.28,...,NaN,NaN,85.55,115.32,100.66,102.45,NaN,NaN,46.26,53.13
08/03,16.25,NaN,NaN,14.71,17.41,24.24,23.56,NaN,NaN,36.59,...,NaN,79.38,86.91,112.51,103.63,NaN,NaN,56.20,47.24,52.68
08/04,16.28,NaN,19.04,14.78,17.65,24.86,NaN,NaN,25.47,35.74,...,125.53,80.21,86.82,106.99,NaN,NaN,105.24,56.71,48.05,53.16
08/05,NaN,16.88,19.14,14.65,17.61,NaN,NaN,23.45,25.67,36.67,...,122.70,81.26,86.40,109.18,NaN,102.29,104.74,56.25,48.05,NaN


In [818]:
Spread_Combi=create_sprd(front_month, back_month)

Please indicate if this is a 1) Crack or 2) Spread - please input integer 1 or 2: 2


In [819]:
Spread_Combi.dropna(axis=0,how='all',inplace=True) # drop rows with all NAs

In [820]:
Spread_Combi.dropna(axis=1,how='all',inplace=True)# drop columns with all NAs

In [821]:
lookback=int(raw_input('Please input the lookback period: '))
df_ma=Spread_Combi
df_std=Spread_Combi



Please input the lookback period: 20


In [822]:
df_ma=pd.rolling_mean(Spread_Combi,lookback)
df_std=pd.rolling_std(Spread_Combi,lookback)

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=20,center=False).mean()
  """Entry point for launching an IPython kernel.
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: pd.rolling_std is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=20,center=False).std()
  


In [823]:
z_score=(Spread_Combi-df_ma)/df_std

In [824]:
z_score_entry=float(raw_input('Please input the z score for trade entry as a positive float number: '))
z_score_exit=float(raw_input('Please input the z score for trade exit as a positive float number: '))

df_long_entry=z_score <= -z_score_entry
df_long_exit=z_score>=-z_score_exit
df_short_entry=z_score>=z_score_entry
df_short_exit=z_score<=z_score_exit

Please input the z score for trade entry as a positive float number: 2.2
Please input the z score for trade exit as a positive float number: 1


In [825]:
rows=Spread_Combi.shape[0]
cols=Spread_Combi.shape[1]

In [826]:
df_long_positions=pd.DataFrame(np.nan,index=range(0,rows),columns=range(0,cols))
df_short_positions=pd.DataFrame(np.nan,index=range(0,rows),columns=range(0,cols))

In [827]:
df_long_entry[df_long_entry==True]=2
df_long_exit[df_long_exit==True]=-1
df_long_positions=df_long_entry+df_long_exit

df_long_positions[df_long_positions==0]=np.nan
df_long_positions[df_long_positions==-1]=0
df_long_positions[df_long_positions==2]=1

df_long_positions=df_long_positions.fillna(method='ffill')

In [828]:
df_short_entry[df_short_entry==True]=-2
df_short_exit[df_short_exit==True]=1
df_short_positions=df_short_entry+df_short_exit

df_short_positions[df_short_positions==0]=np.nan
df_short_positions[df_short_positions==1]=0
df_short_positions[df_short_positions==-2]=-1

df_short_positions=df_short_positions.fillna(method='ffill')

In [829]:
df_positions=df_long_positions+df_short_positions

In [830]:
zf=z_factor(product1, product2)

In [831]:
df_pnl=df_positions.shift(1)*(Spread_Combi*zf[0]-Spread_Combi.shift(1)*zf[1])*zf[2]

In [832]:
df_cumpnl=df_pnl.cumsum()

In [833]:
final_pnl=df_cumpnl.tail(1)

In [834]:
win_percent=float(float((final_pnl[final_pnl>0].count().sum()))/float((final_pnl.count().sum())))*100

In [835]:
print 'The winning percentage is %.2f percent.' %win_percent
print 'The sample size is %.0f.' %final_pnl.count().sum()

The winning percentage is 78.26 percent.
The sample size is 23.


In [836]:
print 'Total pnl over 23 years is $%.2f.'%final_pnl.sum().sum()

Total pnl over 23 years is $24670.00.


In [837]:
z_score_date=raw_input("Please input the date to check in mm/dd format: ")

Please input the date to check in mm/dd format: 05/07


In [838]:
z_score_year=raw_input('Please input the year of the date in which you are in: ')

Please input the year of the date in which you are in: 2018


In [839]:
if z_score[z_score_year].loc[z_score_date]<=-z_score_entry or z_score[z_score_year].loc[z_score_date]>=z_score_entry:
    print "Trade Entry Check!"
    print 'The z-score is %.2f.' %z_score[z_score_year].loc[z_score_date] 
else:
    print 'No entry signal. Please check for exit signal if already in the trade.'
    print 'The z-score is %.2f.' %z_score[z_score_year].loc[z_score_date] 

The z-score is -0.55.
